# SemEval 2019 Task A

In [10]:
import pandas as pd
with open('Subtask-A-master\\Subtask-A-master\\V1.4_Training.csv','r',encoding='utf-8') as file:
    df = pd.read_csv(file,names=['No','Text','Label'],delimiter=',')
    df = df.reset_index(drop=True)
    df = df.set_index('No')

In [11]:
df.head()

,Text,Label
No,,
663_3,"""Please enable removing language code from the...",1
663_4,"""Note: in your .csproj file, there is a Suppor...",0
664_1,"""Wich means the new version not fully replaced...",0
664_2,"""Some of my users will still receive the old x...",0
664_3,"""The store randomly gives the old xap or the n...",0


In [15]:
df.Label.unique()

array([1, 0], dtype=int64)

In [26]:
with open('Subtask-A-master\\Subtask-A-master\\SubtaskA_EvaluationData.csv','r',encoding='utf-8') as f:
    df_test = pd.read_csv(f,names=['No','Text','Label'],delimiter=',')
    df_test = df_test.reset_index(drop=True)
    df_test = df_test.set_index('No')

In [27]:
df_test.head()

,Text,Label
No,,
9566,This would enable live traffic aware apps.,X
9569,Please try other formatting like bold italics ...,X
9576,Since computers were invented to save time I s...,X
9577,Allow rearranging if the user wants to change ...,X
9579,Add SIMD instructions for better use of ARM NE...,X


In [30]:
with open('Subtask-A-master\\Subtask-A-master\\SubtaskA_Trial_Test_Labeled.csv','r',encoding='latin-1') as f:
    df_valid = pd.read_csv(f,names=['No','Text','Label'],delimiter=',')
    df_valid = df_valid.reset_index(drop=True)
    df_valid = df_valid.set_index('No')

In [31]:
df_valid.head()

,Text,Label
No,,
1310_1,I'm not asking Microsoft to Gives permission l...,1
1312_1,somewhere between Android and iPhone.,0
1313_1,And in the Windows Store you can flag the App ...,0
1313_2,"Many thanks Sameh Hi, As we know, there is a l...",0
1313_3,The idea is that we can develop a regular app ...,1


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.Text)
X_train_counts.shape

(8500, 10117)

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(8500, 10117)

In [34]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, df.Label)

In [35]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(df.Text, df.Label)

C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [36]:
import numpy as np
predicted = text_clf.predict(df_valid.Text)
np.mean(predicted == df_valid.Label)

C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


0.5118243243243243

In [40]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(df.Text, df.Label)
predicted_svm = text_clf_svm.predict(df_valid.Text)
np.mean(predicted_svm == df_valid.Label)

C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


0.6114864864864865

In [39]:
# Grid Search
# Here, we are creating a list of parameters for which we would like to do performance tuning. 
# All the parameters name start with the classifier name (remember the arbitrary name we gave). 
# E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [41]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(df.Text, df.Label)

# To see the best mean score and the params, run the following code

print(gs_clf.best_score_)
print(gs_clf.best_params_)

# Output for above should be: The accuracy has now increased to ~90.6% for the NB classifier (not so naive anymore! 😄)
# and the corresponding parameters are {‘clf__alpha’: 0.01, ‘tfidf__use_idf’: True, ‘vect__ngram_range’: (1, 2)}.

0.8104705882352942
{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [42]:
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}

gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(df.Text, df.Label)


print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

0.8325882352941176
{'clf-svm__alpha': 0.001, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [43]:
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem.snowball import SnowballStemmer

text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()), 
                             ('mnb', MultinomialNB(fit_prior=False))])
text_mnb_stemmed = text_mnb_stemmed.fit(df.Text,df.Label)
predicted_mnb_stemmed = text_mnb_stemmed.predict(df_valid.Text)
np.mean(predicted_mnb_stemmed == df_valid.Label)

C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


0.5658783783783784

In [46]:
# Predicting the labels for the test set
# By the choosing the model with the highest prediction score for the labeled set
df_test_dummy = df_test
predicted_test = text_clf_svm.predict(df_test.Text)
for i in range((df_test.Text.count())):
    df_test['Label'].iloc[i] = predicted_test[i]


C:\Users\RISHAV\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [47]:
df_test

,Text,Label
No,,
9566,This would enable live traffic aware apps.,0
9569,Please try other formatting like bold italics ...,0
9576,Since computers were invented to save time I s...,0
9577,Allow rearranging if the user wants to change ...,0
9579,Add SIMD instructions for better use of ARM NE...,0
9580,Also using a hot swapping code generator (opti...,0
9587,Microsoft should seriously look into getting r...,0
9599,I would be extremely useful in a variety of ap...,1
9626,I would like to see informatin about the certi...,1
